In [468]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [469]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [471]:
import re
def get_title(name):
    if pd.isnull(name):
        return 'Null'
    title_search = re.search('([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1).lower()
    else:
        return 'None'

In [472]:
titles = {'mr':1, 'mrs':2, 'mme':2, 'ms':3, 'miss':3, 'mlle':3, 'don':4, 'sir':4, 'jonkheer':4,
          'major':4, 'col':4, 'dr':4, 'master':4, 'capt':4, 'dona':5, 'lady':5, 'countess':5,
         'rev':6}
df_train['Title'] = df_train['Name'].apply(lambda name: titles.get(get_title(name)))
df_train['Honor'] = df_train['Title'].apply(lambda title: 1 if title ==4 or title == 5 else 0)
df_test['Title'] = df_test['Name'].apply(lambda name: titles.get(get_title(name)))
df_test['Honor'] = df_test['Title'].apply(lambda title: 1 if title ==4 or title == 5 else 0)

In [474]:
# for i in range(1,7):
#     print('Title: {0:}, mean: {1:}'.format(i,df_train.Age[df_train.Title==i].median()))

# for i in range(1,7):
#     per = df_train[df_train.Title==i].Survived.sum()/df_train.Age[df_train.Title==i].shape[0]*100
#     print('Title: {0:}, survived: {1:}, per: {2:.2f}%'.format(i, df_train[df_train.Title==i].Survived.sum(), per))

# a = df_train[(df_train.Age.isnull()) & (df_train.Title==4)]
# b = df_train[(df_train.Age.isnull()) & (df_train.Title==2)]
# b.index

In [ ]:
# 1. Data clearning
# Age - find the median and fill to 
# may be try df_test_Age with its own median
# fill_df_test_Age = df_test.Age.median()
# fill_df_Age = df_train.Age.median()
# df_train.Age.fillna(fill_df_Age, inplace=True)
# df_test.Age.fillna(fill_df_Age, inplace=True)
for i in range(1,7):
    a = df_train[(df_train.Age.isnull()) & (df_train.Title==i)]
    df_train.Age.iloc[a.index] = df_train.Age[df_train.Title==i].median()
    b = df_test[(df_test.Age.isnull()) & (df_test.Title==i)]
    df_test.Age.iloc[b.index] = df_train.Age[df_train.Title==i].median()

# Cabin - fill nan with 'fill' and select first cabin
df_train.Cabin.fillna('fill',inplace=True)
df_train.Cabin = df_train.Cabin.map(lambda x:x.split(' ')[0])
df_test.Cabin.fillna('fill', inplace=True)
df_test.Cabin = df_test.Cabin.map(lambda x:x.split(' ')[0])

# Sex - male: 1, female: 1
df_train.Sex.replace({'male':1, 'female':0}, inplace=True)
df_test.Sex.replace({'male':1, 'female':0}, inplace=True)

df_train = df_train.dropna(axis=0, how='any')
df_test.Fare.fillna(df_test.Fare.mean(), inplace=True)
# df_test = df_test.dropna(axis=0, how='any')

In [477]:
df_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Title', 'Honor'],
      dtype='object')

In [458]:
# 2. Select the features
sFeatures = ['Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',             
 'Cabin',             
 'Embarked',
 'Title',
 'Honor']

X = df_train[sFeatures]
y = df_train['Survived']
X_test = df_test[sFeatures]

In [459]:
# 3. Label Encoder
# X_test has elements that X doesn't have
# create a Cabin_labels cover X and X_test
X_Cabin_unique = X['Cabin'].unique()
X_lack = X_test.Cabin[X_test['Cabin'].isin(X_Cabin_unique)==0].values
Cabin_labels = np.append(X_Cabin_unique, X_lack)
# print(X_Cabin_unique.shape, X_Cabin_unique)
# print(Cabin_labels.shape, Cabin_labels)

le_Cabin = LabelEncoder().fit(Cabin_labels)
le_Embarked = LabelEncoder().fit(np.array(X['Embarked'].tolist()))

X.loc[:,'Cabin'] = le_Cabin.transform(X['Cabin'])
X.loc[:,'Embarked'] = le_Embarked.transform(X['Embarked'])
X_test.loc[:,'Cabin'] = le_Cabin.transform(X_test['Cabin'])
X_test.loc[:,'Embarked'] = le_Embarked.transform(X_test['Embarked'])

In [478]:
X.dtypes

Pclass        int64
Sex           int64
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Cabin         int64
Embarked      int64
Title         int64
Honor         int64
dtype: object

In [461]:
# Validation
# X_train, X_test, y_train, y_test = train_test_split(X, y,)

### Tensor Graph

In [462]:
# #
# features= 8
# X_train = tf.placeholder(tf.float32, shape=[None,features
# y_train = tf.placeholder(tf.float32, shape=[None,2])
# W = tf.Variable(tf.random_normal([features, 2]), name='weights')
# b = tf.Variable(tf.zeros(2), name='bias')
# y_pred = tf.nn.softmax(tf.matmul(input, W) + bias)

In [463]:
# # Cross Entropy
# learning_rate = 0.001
# cross_entropy = -tf.reduce_sum(y * tf.log(y_pred + 1e-10), axis=1)
# cost = tf.reduce_mean(cross_entropy)
# train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [465]:
# scaler = MinMaxScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)
X_train = X
y_train = y

In [466]:
X_train[0:10]

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title,Honor
0,3,1,22.0,1,0,7.2500,180,2,1,0
1,1,0,38.0,1,0,71.2833,105,0,2,0
2,3,0,26.0,0,0,7.9250,180,2,3,0
3,1,0,35.0,1,0,53.1000,69,2,2,0
4,3,1,35.0,0,0,8.0500,180,2,1,0
5,3,1,30.0,0,0,8.4583,180,1,1,0
6,1,1,54.0,0,0,51.8625,162,2,1,0
7,3,1,2.0,3,1,21.0750,180,2,4,1
8,3,0,27.0,0,2,11.1333,180,2,2,0
9,2,0,14.0,1,0,30.0708,180,0,2,0


KNN

Decision Tree

Logistic Regression

In [ ]:
# Logistic Regression
lr = LogisticRegression(C=10, max_iter=100, solver='newton-cg').fit(X_train, y_train)
yp = lr.predict(X_train)
y_predict = lr.predict(X_test)

passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
submission.to_csv('submission_lr.csv')
# passengerid = np.array(df_train.PassengerId.tolist())
# submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': yp}).set_index('PassengerId')
# submission.to_csv('y_train_predict.csv')

Gradient Boosting Classifier

In [ ]:
# Gradient Boosting Classifier
# gbdt = GradientBoostingClassifier(n_estimators=300, max_depth=5).fit(X_train, y_train)
# y_predict = gbdt.predict(X_test)

# passengerid = np.array(df_test.PassengerId.tolist())
# submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
# submission.to_csv('submission.csv')

Random forests

In [467]:
# RandomForest
rfc = RandomForestClassifier(n_estimators=200, max_depth=5).fit(X_train, y_train)
y_predict = rfc.predict(X_test)
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
submission.to_csv('submission_random_3.csv')

Stochastic Gradient Descent (SDG)

In [ ]:
# Stochastic Gradient Descent (SGD) easily scale with more than 10^5 samples and features
sgd = SGDClassifier(alpha=0.001, max_iter=1000).fit(X_train, y_train)
y_predict = sgd.predict(X_test)
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
submission.to_csv('submission_sdg_2.csv')

Navie Bayes – GaussianNB

In [ ]:
Dummy Classifier